In [ ]:
from __future__ import division
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sci
from numba import jit

# Here is an implementation of contineous and discrete cases. 
## In "statistical propagator" you can choose "old=True" to get the old NDM. 
I don't have the Jacobians implementation in my codes, but they are easy to plug in. You need to replace the Laplacian with the Jacobian of your choice using the table in PRE.

In [ ]:
def H_diff_continuous(g):
    # Calculate the diffusion control operator.
    A = nx.to_numpy_array(g)
    return np.sum(A,axis=0)*np.eye(len(A)) - A

def H_rw_discrete(g):
    # Calculate the random walk control operator (transition matrix).
    A = nx.to_numpy_array(g)
    np.fill_diagonal(A, 1) # adding self-loops is a typical practice to avoid problems with isolated nodes.
    K_inv = (1/np.sum(A,axis=0))*np.eye(len(A))
    return A @ K_inv

def G_tau_continuous(H,tau):
    # Calculate the time-evolution operator for continuous dynamics
    # H is the control operator
    # tau is the propagation time
    return sci.linalg.expm(-tau * H)

def G_tau_discrete(H,tau):
    # Calculate the time-evolution operator for discrete dynamics
    # H is the control operator
    # tau is the propagation time
    return H**tau

def statistical_propagator(H , tau , continuous = True  ,old = False):
    ### Returns {density matrix, entropy, partition function, free energy}
    if continuous:
        G_tau = G_tau_continuous(H,tau)
    else:
        G_tau = G_tau_discrete(H,tau)
    ### With the assumption that all nodes have equal probability of perturbation 
    if old:
        U = G_tau / len(H)
    else:
        U = G_tau @ G_tau.T / len(H)
    return U

def network_thermodynamics(H, tau, continuous = True):
    U_t = statistical_propagator( H , tau , continuous = True  )
    Z = np.trace(U_t)  # partition function
    rho = U_t/Z  # density matrix

    ### Make sure the numerical errors doesn't affect entropy
    eigvals, eigvecs = np.linalg.eigh(rho)
    eigvals = np.delete(eigvals, np.where(abs(eigvals)<10**-10)).real
    #######################
    
    eigvals /= np.sum(eigvals)
    S = -np.sum( eigvals * np.log(eigvals)) # entropy
    
    return rho, S, Z, -np.log(Z)/tau

@jit(nopython=True)
def H_J(coupling1, coupling2, angles_vec1, angles_vec2, adj_mat1, adj_mat2, omega_f, A, B, t):
    N1 = len(adj_mat1)
    N2 = len(adj_mat2)
    J = np.zeros((N1+N2+1, N1+N2+1))
    for i in range(N1):
        for j in range(N1):
            if i == j:
                for k in range(N1):
                    if k!=i:
                        J[i,j] -= coupling1*adj_mat1[i,k]*np.cos(angles_vec1[k] - angles_vec1[i])
            else:
                J[i,j] = coupling1*adj_mat1[i,j]* np.cos(angles_vec1[j] - angles_vec1[i])
    for i in range(N2):
        for j in range(N2):
            if i == j:
                for k in range(N2):
                    if k!=i:
                        J[N1+i,N1+j] -= coupling2*adj_mat2[i,k]*np.cos(angles_vec2[k] - angles_vec2[i])
            else:
                J[N1+i,N1+j] = coupling2*adj_mat2[i,j]* np.cos(angles_vec2[j] - angles_vec2[i])
    for i in range(N1):
        J[N1+N2, i] = -A*np.abs(1j*np.exp(1j*angles_vec1[i]))/N1 + B*np.abs(1j*np.sum(np.exp(1j*(angles_vec1[i] - angles_vec2[j])) for j in range(N2)))/(N1*N2)
        J[i, N1+N2] = np.sin(omega_f*t - angles_vec1[i])
    for i in range(N2):
        J[N1+N2, N1+i] = -A*np.abs(1j*np.exp(1j*angles_vec2[i]))/N2 + B*np.abs(-1j*np.sum(np.exp(1j*(angles_vec1[j] - angles_vec2[i])) for j in range(N1)))/(N1*N2)
        J[N1+i, N1+N2] = np.sin(omega_f*t - angles_vec2[i])
        
    return -J

# You need to choose a network (g) and a control operator (H)
## I have built the control operators for continuous diffusion and discrete random walks as examples
You can plug in the Jacobian instead of Laplacian--- and make it a new function.

In [ ]:
g = nx.erdos_renyi_graph(30,.2)
H = H_J(0.2, 0.0, )
tau = 3
rho , S , Z , F= network_thermodynamics(H, tau, continuous = True)

In [9]:
S

np.float64(0.0571844667445924)

In [121]:
g = nx.erdos_renyi_graph(30,.2)
H = H_rw_discrete(g)
tau = 3
rho , S , Z = network_thermodynamics(H, tau, continuous = False)